## Pre-processing and Visualization

In [1]:
# Import the required libraries.
import re
import math
import random
import collections
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

from collections import defaultdict

import operator

random.seed(11)
np.random.seed(11)

In [2]:
# unique_tags_count_dict = dict()
# word_tag_dict = dict()
# tag_tag_dict = dict()

# def parse_sentence(sentence):
#     previous = "<s>"
#     previous = previous.strip()
#     if previous not in unique_tags_count_dict:
#         unique_tags_count_dict[previous] = 0
#     unique_tags_count_dict[previous] += 1

#     word_tags = sentence.split(" ")

#     for i, word_tag in enumerate(word_tags):
#         word_tag = word_tag.strip()
#         var_array = word_tag.split("/")
#         tag = var_array[-1]
#         tag = tag.strip()

#         if tag not in unique_tags_count_dict:
#             unique_tags_count_dict[tag] = 0
#         unique_tags_count_dict[tag] += 1

#         if word_tag not in word_tag_dict:
#             word_tag_dict[word_tag] = 0
#         word_tag_dict[word_tag] += 1

#         if previous + "/" + tag not in tag_tag_dict:
#             tag_tag_dict[previous + "/" + tag] = 0
#         tag_tag_dict[previous + "/" + tag] += 1

#         previous = tag

#     if previous + "/<~s>" not in tag_tag_dict:
#         tag_tag_dict[previous + "/<~s>"] = 0
#     tag_tag_dict[previous + "/<~s>"] += 1

#     return 

In [3]:
def parse_sentence(sentence):
    word_tag_pairs = sentence.split(" ")
    words = []
    tags = []
    for i, word_tag in enumerate(word_tag_pairs):
        word, tag = word_tag.strip().rsplit('/', 1)
        words.append(word)
        tags.append(tag)
        
    return words, tags

In [4]:
parsed_sentences = []
with open('./Brown_train.txt', 'r') as file:
    sentences = file.readlines()
    for sentence in sentences:
        sentence = sentence.strip()
        parsed_sentences.append(parse_sentence(sentence))

In [5]:
kf = KFold(n_splits=3, shuffle=False)
parsed_sentences = np.asarray(parsed_sentences)
for train_index, test_index in kf.split(parsed_sentences):
    train_data = parsed_sentences[train_index]
    test_data = parsed_sentences[test_index]
    X_train = [a[0] for a in train_data]
    Y_train = [a[1] for a in train_data]
    X_test = [a[0] for a in test_data]
    Y_test = [a[1] for a in test_data]
    
    break

In [6]:
def get_vocab(X_train, Y_train):
    vocabulary2id = dict()
    
    tag2id = dict()
    vocabulary2id['UNK'] = 0
    for sent in X_train:
        for word in sent:
            if word not in vocabulary2id.keys():
                vocabulary2id[word] = len(vocabulary2id)
    for sent in Y_train:
        for tag in sent:
            if tag not in tag2id.keys():
                tag2id[tag] = len(tag2id)
    
    return vocabulary2id, tag2id

def get_word_tag_counts(X_train, Y_train, vocabulary2id, tag2id):
    wordcount = defaultdict(int)
    tagcount = defaultdict(int)
    tagpaircount = defaultdict(int)
    tagtriplecount = defaultdict(int)
    
    for sent in X_train:
        for word in sent:
            wordcount[word] += 1
    
    for sent in Y_train:
        for tag in sent:
            tagcount[tag] += 1
    
    for sent in Y_train:
        for i in range(len(sent) - 1):
            tagpaircount[sent[i], sent[i + 1]] += 1

    for sent in Y_train:
        for i in range(len(sent) - 2):
            tagtriplecount[sent[i], sent[i + 1], sent[i + 2]] += 1
    
    return wordcount, tagcount, tagpaircount, tagtriplecount


In [8]:
vocabulary2id, tag2id = get_vocab(X_train, Y_train)
wordcount, tagcount, tagpaircount, tagtriplecount = get_word_tag_counts(X_train, Y_train, vocabulary2id, tag2id)

In [80]:
UNK = "UNK"  # token to map all out-of-vocabulary words (OOVs)
UNKid = 0      # index for UNK
epsilon=1e-100
array, ones, zeros, multiply, unravel_index = np.array, np.ones, np.zeros, np.multiply, np.unravel_index

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, 
                 smoothing_obs = 0.01, 
                 transition_proba1= None,
                 prob_abs= 0.00001):
            """
            Builds a Hidden Markov Model
            * state_list is the list of state symbols [q_0...q_(N-1)]
            * observation_list is the list of observation symbols [v_0...v_(M-1)]
            * transition_proba is the transition probability matrix
                [a_ij] a_ij,a_ik = Pr(Y_(t+1)=q_i|Y_t=q_j,Y_(t-1)=q_k)
            * observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            * initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            self.prob_abs = prob_abs
            
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba1 is None:
                self.transition_proba1 = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba1=transition_proba1
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
        
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
        
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices

        def data2indices(self, sent): 
            """From one tagged sentence of the brown corpus: 
            - extract the words and tags 
            - returns two list of indices, one for each
            -> (wordids, tagids)
            """
            wordids = list()
            tagids  = list()
            for couple in sent:
                wrd = couple[0]
                tag = couple[1]
                if wrd in self.X_index:
                    wordids.append(self.X_index[wrd])
                else:
                    wordids.append(UNKid)
                tagids.append(self.Y_index[tag])
            return wordids,tagids
        
        def observation_estimation(self, pair_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            # fill with counts
            for pair in pair_counts:
                wrd=pair[0]
                tag=pair[1]
                cpt=pair_counts[pair]
                k = 0 # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                i=self.Y_index[tag]
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)

        def transition_estimation(self, trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j,Y_(t-1)=q_k)
            """
            # fill with counts
            for triple in trans_counts:
                i=self.Y_index[triple[2]]
                j=self.Y_index[triple[1]]
                k=self.Y_index[triple[0]]
                self.transition_proba[k,j,i]=trans_counts[triple]
            # normalize sorun cıkacak !!!
            self.transition_proba=self.transition_proba/self.transition_proba.sum(axis=0).reshape(self.N,self.N)

        def transition_estimation1(self, trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
            """
            # fill with counts
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index[pair[0]]
                self.transition_proba1[j,i]=trans_counts[pair]
            # normalize
            self.transition_proba1=self.transition_proba1/self.transition_proba1.sum(axis=0).reshape(1,self.N)
        
        
        def init_estimation(self, init_counts):
            """Build the init. distribution"""
            # fill with counts
            for tag in init_counts:
                i=self.Y_index[tag]
                self.initial_state_proba[i]=init_counts[tag]
            # normalize
            self.initial_state_proba=self.initial_state_proba/sum(self.initial_state_proba)
             
        
        def supervised_training(self, pair_counts, trans_counts,init_counts,trans_counts1):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.transition_estimation1(trans_counts1)
            self.init_estimation(init_counts)
        
        def viterbi(self,observations):
            if len(observations)<2:
                return [hmm.Y_index[z] for z in observations]
            nSamples = len(observations)
            nStates = self.transition_proba.shape[0] # number of states
            c = np.zeros(nSamples) #scale factors (necessary to prevent underflow)
            viterbi = np.zeros((nStates,nStates,nSamples)) # initialise viterbi table
            viterbi1 = np.zeros((nStates,nSamples)) # initialise viterbi table
            psi = np.zeros((nStates,nStates,nSamples)) # initialise the best path table
            best_path = np.zeros(nSamples); # this will be your output
            idx0 = self.X_index[observations[0]]
            idx1 = self.X_index[observations[1]]
            viterbi1[:,0] = self.initial_state_proba.T * self.observation_proba[idx0,:].T

            for s in range (0,nStates): # loop through the states @(t-2)
                for v in range (0,nStates): # loop through the states @(t-1)
                    viterbi[s,v,1] = viterbi1[s,0] * self.transition_proba1[s,v] * self.observation_proba[idx1,v]

            psi[0] = 0;

            for t in range(2,nSamples): # loop through time
                idx = self.X_index[observations[t]]
                for s in range (0,nStates): # loop through the states @(t-1)
                    for v in range (0,nStates): # loop through the states @(t-1)
                        self.transition_proba[np.isnan(self.transition_proba)] = self.prob_abs
                        trans_p = viterbi[:,s,t-1] * self.transition_proba[:,s,v]
                
                        if(math.isnan(trans_p[0])):
                            trans_p[0]=0

                        psi[s,v,t], viterbi[s,v,t] = max(enumerate(trans_p), key=operator.itemgetter(1))
                        viterbi[s,v,t] = viterbi[s,v,t]*self.observation_proba[idx,v]


            cabbar = viterbi[:,:,nSamples-1]
            best_path[nSamples-1] = unravel_index(cabbar.argmax(), cabbar.shape)[1]
            best_path[nSamples-2] = unravel_index(cabbar.argmax(), cabbar.shape)[0]

#             return best_path, nSamples, psi
            for t in range(nSamples-3,-1,-1): # states of (last-1)th to 0th time step
                best_path[t] = psi[int(round(best_path[t+1])), int(round(best_path[t+2])), t+2]

            return best_path
        
        def fwd_bkw(self, observations):
            observations = x
            self = hmm

            nStates = self.transition_proba.shape[0]
            start_prob = self.initial_state_proba
            trans_prob = self.transition_proba1.transpose()

            emm_prob = self.observation_proba.transpose()

            # forward part of the algorithm
            fwd = []
            f_prev = {}
            for i, observation_i in enumerate(observations):
                f_curr = {}
                for st in range(nStates):
                    if i == 0:
                        # base case for the forward part
                        prev_f_sum = start_prob[st]
                    else:
                        prev_f_sum = sum(f_prev[k] * trans_prob[k][st] for k in range(nStates))

                    f_curr[st] = emm_prob[st][self.X_index[observation_i]] * prev_f_sum

                fwd.append(f_curr)
                f_prev = f_curr

            p_fwd = sum(f_curr[k] for k in range(nStates))

            # backward part of the algorithm
            bkw = []
            b_prev = {}
            for i, observation_i_plus in enumerate(reversed(observations[1:] + [None,])):
                b_curr = {}
                for st in range(nStates):
                    if i == 0:
                        # base case for backward part
                        b_curr[st] = 1.0
                    else:
                        b_curr[st] = sum(trans_prob[st][l] * emm_prob[l][self.X_index[observation_i_plus]] * b_prev[l] for l in range(nStates))

                bkw.insert(0,b_curr)
                b_prev = b_curr

            p_bkw = sum(start_prob[l] * emm_prob[l][self.X_index[observations[0]]] * b_curr[l] for l in range(nStates))

            # merging the two parts
            posterior = []
            for i in range(len(observations)):
                posterior.append({st: fwd[i][st] * bkw[i][st] / p_fwd for st in range(nStates)})

            assert abs(p_fwd - p_bkw) < 1e-6
            return fwd, bkw, posterior

In [81]:
def make_counts(X, Y):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns: 
    * c_words: word counts
    * c_tags: tag counts
    * c_pairs: count of pairs (word,tag)
    * c_transitions: count of tag bigram 
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_inits = dict()
    c_transitions1 = dict()
    
    for sent in zip(X, Y):
        # we use i because of the transition counts
        sent = list(zip(*sent))
        for i in range(len(sent)):
            couple = sent[i]
            wrd = couple[0]
            tag = couple[1]
            # word counts
            if wrd in c_words:
                c_words[wrd]=c_words[wrd]+1
            else:
                c_words[wrd]=1
            # tag counts
            if tag in c_tags:
                c_tags[tag]=c_tags[tag]+1
            else:
                c_tags[tag]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            # j never is seen at second position
            if i >= 1:
                trans1 = (sent[i-1][1],tag)
                if trans1 in c_transitions1:
                    c_transitions1[trans1]=c_transitions1[trans1]+1
                else:
                    c_transitions1[trans1]=1
            
            if i > 1:
                trans = (sent[i-2][1],sent[i-1][1],tag)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
            # i == 0 -> counts for initial states
            else:
                if tag in c_inits:
                    c_inits[tag]=c_inits[tag]+1
                else:
                    c_inits[tag]=1
    
    return c_words,c_tags,c_pairs, c_transitions, c_inits, c_transitions1

In [82]:
cwords, ctags, cpairs, ctrans, cinits, ctrans1 = make_counts(X_train, Y_train)

In [83]:
state_list = list(ctags.keys())
observation_list = [a[0] for a in sorted(vocabulary2id.items(), key=lambda x: x[1])]
hmm = HMM(state_list=state_list, observation_list=observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                 smoothing_obs = 0.4,
                 prob_abs = 0)
hmm.supervised_training(cpairs,ctrans,cinits,ctrans1)

/home/stud/btech/cse/2016/mukuntha.cs16/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:123: RuntimeWarning: invalid value encountered in true_divide


In [228]:
scores = []

for x, y_true in zip(X_test, Y_test):
    for i in range(len(x)):
        if x[i] not in vocabulary2id.keys():
            x[i] = 'UNK'
    pred_idx = hmm.viterbi(x)
    y_pred = np.asarray([state_list[int(round(i))] for i in pred_idx])
    y_true = np.asarray(y_true)
    scores += (y_pred == y_true).tolist()

In [229]:
print('Accuracy: {}'.format(np.asarray(scores).mean()))

Accuracy: 0.8308295872496935


In [84]:
x, y_true = X_train[0], Y_train[0]

In [107]:
scores = []

for x, y_true in zip(X_test, Y_test):
    for i in range(len(x)):
        if x[i] not in vocabulary2id.keys():
            x[i] = 'UNK'
    pred_probs = hmm.fwd_bkw(x)
    pred_idx = [max(probs.items(), key=lambda x: x[1])[0] for probs in pred_probs[2]]
    y_pred = np.asarray([state_list[int(round(i))] for i in pred_idx])
    y_true = np.asarray(y_true)
    scores += (y_pred == y_true).tolist()

/home/stud/btech/cse/2016/mukuntha.cs16/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:249: RuntimeWarning: invalid value encountered in double_scalars


In [108]:
print('Accuracy with only forward-backward: {}'.format(np.asarray(scores).mean()))

Accuracy with only forward-backward: 0.7162525541479362
